# Feature Engineering - Applying EDA Learnings

**Project**: Loan Approval Decision Support System  
**Phase**: 3 - Feature Engineering  
**Date**: February 14, 2026

## Objectives

This notebook applies insights from the EDA (Phase 2) to create predictive features for mortgage approval modeling:

1. **Create Underwriting Metrics**: LTV, Loan-to-Income, Housing Expense Ratio
2. **Engineer Risk Indicators**: DTI risk flags, combined risk scores
3. **Geographic Features**: Target encoding for high-cardinality location features
4. **Interaction Features**: DTI × LTV, Income × Location
5. **Feature Selection**: Remove redundant/low-importance features
6. **Final Preprocessing**: Scaling, train-test split preparation

## Key EDA Insights Applied

- ✅ **DTI is primary predictor**: Create 4-tier risk classification
- ✅ **Co-borrower impact**: 92% approval vs 88% for solo applicants
- ✅ **Geographic variation**: State-level approval rates vary 78-94%
- ✅ **Income-Loan relationship**: Strong correlation (0.7+), create ratio features
- ✅ **Age patterns**: Middle-aged applicants show highest approval rates

---

In [15]:
# ============================================================================
# IMPORT LIBRARIES
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Sklearn imports
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif, SelectKBest, chi2
from scipy.stats import chi2_contingency

# Configure visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [16]:
# ============================================================================
# SET WORKING DIRECTORY
# ============================================================================
import os

# Set working directory to project root
project_root = Path('/Users/josiahgordor/Desktop/DSPortfolio/Projects/loan_approval')
os.chdir(project_root)

print("="*80)
print("WORKING DIRECTORY VERIFICATION")
print("="*80)
print(f"Current working directory: {os.getcwd()}")
print(f"✅ Working directory set to project root")

WORKING DIRECTORY VERIFICATION
Current working directory: /Users/josiahgordor/Desktop/DSPortfolio/Projects/loan_approval
✅ Working directory set to project root


In [17]:
# ============================================================================
# LOAD CLEANED DATA
# ============================================================================

# Load data from Phase 2 (Data Cleaning)
data_path = Path('data/processed/loan_data_cleaned_v2.csv')

print(f"Loading data from: {data_path}")
df = pd.read_csv(data_path, low_memory=False)

print("="*80)
print("DATA LOADED")
print("="*80)
print(f"Shape: {df.shape[0]:,} applications × {df.shape[1]} features")
print(f"Target: {df['target'].mean():.2%} approval rate")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display first few rows
df.head(3)

Loading data from: data/processed/loan_data_cleaned_v2.csv
DATA LOADED
Shape: 493,568 applications × 78 features
Target: 89.60% approval rate
Memory: 1794.02 MB


,unnamed:_0,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,lien_status,total_units,conforming_loan_limit,...,tract_median_age_of_housing_units,target,loan_type_Federal Housing Administration insured (FHA),loan_type_USDA Rural Housing Service or Farm Service Agency guaranteed (RHS or FSA),loan_type_Veterans Affairs guaranteed (VA),construction_method_Site-Built,dti_risk_flag,has_coborrower,race_minority,hispanic
0,0,2024,549300DD4R4SYK5RAQ92,34980,0TN,47043.0,47043060502,First Lien,Single Family (1-4 Units),Conforming,...,48,1.0,0,0,0,1,Elevated Risk,True,0,0
1,1,2024,549300DD4R4SYK5RAQ92,99999,0WV,54109.0,54109002902,First Lien,Single Family (1-4 Units),Conforming,...,38,0.0,0,1,0,0,Moderate Risk,True,0,0
2,3,2024,549300DD4R4SYK5RAQ92,24660,0NC,37081.0,37081017102,First Lien,Single Family (1-4 Units),Conforming,...,41,1.0,0,0,0,1,Low Risk,True,0,0


## 1. Create Underwriting Metrics

Based on EDA insights, we'll create key financial ratios used in mortgage underwriting.

In [18]:
# ============================================================================
# TASK 3.1: CREATE UNDERWRITING METRICS
# ============================================================================

print("="*80)
print("CREATING UNDERWRITING METRICS")
print("="*80)

# Create a copy for feature engineering
df_fe = df.copy()

# 1. Loan-to-Income Ratio (LTI)
df_fe['loan_to_income_ratio'] = df_fe['loan_amount'] / df_fe['income']
df_fe['loan_to_income_ratio'] = df_fe['loan_to_income_ratio'].clip(upper=10)  # Cap extreme values
print(f"✓ Loan-to-Income Ratio: Created (Median: {df_fe['loan_to_income_ratio'].median():.2f})")

# 2. Loan-to-Value Ratio (LTV)
if 'property_value' in df_fe.columns:
    df_fe['loan_to_value_ratio'] = (df_fe['loan_amount'] / df_fe['property_value']) * 100
    df_fe['loan_to_value_ratio'] = df_fe['loan_to_value_ratio'].clip(upper=150)  # Cap at 150%
    print(f"✓ Loan-to-Value Ratio (LTV): Created (Median: {df_fe['loan_to_value_ratio'].median():.1f}%)")
else:
    print("⚠ Property value not available - LTV not created")

# 3. Housing Expense Ratio
if 'interest_rate' in df_fe.columns:
    # Monthly payment estimate: P = L[c(1+c)^n]/[(1+c)^n-1]
    # Simplified: use loan amount * interest rate / 12 / income as proxy
    df_fe['housing_expense_ratio'] = (df_fe['loan_amount'] * df_fe['interest_rate'] / 100 / 12) / (df_fe['income'] / 12)
    df_fe['housing_expense_ratio'] = df_fe['housing_expense_ratio'].clip(upper=1)  # Cap at 100%
    print(f"✓ Housing Expense Ratio: Created (Median: {df_fe['housing_expense_ratio'].median():.3f})")
else:
    print("⚠ Interest rate not available - Housing Expense Ratio not created")

# 4. Total Loan Costs Ratio
if 'total_loan_costs' in df_fe.columns:
    df_fe['loan_costs_ratio'] = df_fe['total_loan_costs'] / df_fe['loan_amount']
    df_fe['loan_costs_ratio'] = df_fe['loan_costs_ratio'].clip(upper=0.2)  # Cap at 20%
    print(f"✓ Loan Costs Ratio: Created (Median: {df_fe['loan_costs_ratio'].median():.3f})")
else:
    print("⚠ Total loan costs not available - Loan Costs Ratio not created")

# Display summary statistics
new_features = ['loan_to_income_ratio']
if 'loan_to_value_ratio' in df_fe.columns:
    new_features.append('loan_to_value_ratio')
if 'housing_expense_ratio' in df_fe.columns:
    new_features.append('housing_expense_ratio')
if 'loan_costs_ratio' in df_fe.columns:
    new_features.append('loan_costs_ratio')

print("\n" + "="*80)
print("NEW UNDERWRITING METRICS SUMMARY")
print("="*80)
print(df_fe[new_features].describe())

# Check correlation with target
print("\n" + "="*80)
print("CORRELATION WITH APPROVAL (TARGET)")
print("="*80)
for feat in new_features:
    corr = df_fe[[feat, 'target']].corr().iloc[0, 1]
    print(f"{feat:30s}: {corr:+.4f}")

CREATING UNDERWRITING METRICS
✓ Loan-to-Income Ratio: Created (Median: 2.82)
✓ Loan-to-Value Ratio (LTV): Created (Median: 90.2%)
✓ Housing Expense Ratio: Created (Median: 0.174)
✓ Loan Costs Ratio: Created (Median: 0.018)

NEW UNDERWRITING METRICS SUMMARY
       loan_to_income_ratio  loan_to_value_ratio  housing_expense_ratio  \
count         493568.000000        493568.000000          493568.000000   
mean               2.865843            81.909155               0.165187   
std                1.322241            22.953015               0.093725   
min                0.000112             0.775194               0.000000   
25%                2.037037            77.464789               0.111513   
50%                2.824037            90.243902               0.174047   
75%                3.646617            97.101449               0.229115   
max               10.000000           150.000000               1.000000   

       loan_costs_ratio  
count     493568.000000  
mean           

## 2. Create Risk Indicators

Categorical risk flags based on lending industry standards and EDA insights.

In [19]:
# ============================================================================
# TASK 3.2: CREATE RISK INDICATORS
# ============================================================================

print("="*80)
print("CREATING RISK INDICATORS")
print("="*80)

# 1. DTI Risk Flag (from EDA - if not already present)
if 'dti_risk_flag' not in df_fe.columns and 'debt_to_income_ratio' in df_fe.columns:
    # Map DTI categories to risk levels (from EDA analysis)
    dti_risk_mapping = {
        '<20%': 'Low Risk',
        '20%-<30%': 'Low Risk',
        '30%-<36%': 'Moderate Risk',
        '36': 'Moderate Risk',
        '37': 'Moderate Risk',
        '38': 'Moderate Risk',
        '39': 'Moderate Risk',
        '40': 'Moderate Risk',
        '41': 'Moderate Risk',
        '42': 'Moderate Risk',
        '43': 'Moderate Risk',
        '44': 'Elevated Risk',
        '45': 'Elevated Risk',
        '46': 'Elevated Risk',
        '47': 'Elevated Risk',
        '48': 'Elevated Risk',
        '49': 'Elevated Risk',
        '50%-60%': 'Elevated Risk',
        '>60%': 'High Risk',
        'Exempt': 'Unknown'
    }
    
    df_fe['dti_risk_flag'] = df_fe['debt_to_income_ratio'].astype(str).map(dti_risk_mapping)
    df_fe['dti_risk_flag'] = df_fe['dti_risk_flag'].fillna('Unknown')
    print(f"✓ DTI Risk Flag created: {df_fe['dti_risk_flag'].nunique()} categories")
    print(df_fe['dti_risk_flag'].value_counts())
else:
    print("✓ DTI Risk Flag already exists or DTI not available")

# 2. LTV Risk Flag
if 'loan_to_value_ratio' in df_fe.columns:
    def ltv_risk_category(ltv):
        if pd.isna(ltv):
            return 'Unknown'
        elif ltv <= 80:
            return 'Low Risk'
        elif ltv <= 90:
            return 'Moderate Risk'
        elif ltv <= 97:
            return 'Elevated Risk'
        else:
            return 'High Risk'
    
    df_fe['ltv_risk_flag'] = df_fe['loan_to_value_ratio'].apply(ltv_risk_category)
    print(f"\n✓ LTV Risk Flag created: {df_fe['ltv_risk_flag'].nunique()} categories")
    print(df_fe['ltv_risk_flag'].value_counts())
else:
    print("\n⚠ LTV not available - LTV Risk Flag not created")

# 3. Income Bracket (for stratification)
if 'income' in df_fe.columns:
    income_brackets = [0, 50000, 75000, 100000, 150000, 250000, np.inf]
    income_labels = ['<$50K', '$50-75K', '$75-100K', '$100-150K', '$150-250K', '>$250K']
    df_fe['income_bracket'] = pd.cut(df_fe['income'], bins=income_brackets, labels=income_labels)
    print(f"\n✓ Income Bracket created: {df_fe['income_bracket'].nunique()} categories")
    print(df_fe['income_bracket'].value_counts())
else:
    print("\n⚠ Income column not available - Income Bracket not created")

# 4. Loan Size Category
loan_brackets = [0, 200000, 300000, 400000, 600000, np.inf]
loan_labels = ['<$200K', '$200-300K', '$300-400K', '$400-600K', '>$600K']
df_fe['loan_size_category'] = pd.cut(df_fe['loan_amount'], bins=loan_brackets, labels=loan_labels)
print(f"\n✓ Loan Size Category created: {df_fe['loan_size_category'].nunique()} categories")
print(df_fe['loan_size_category'].value_counts())

# 5. Combined Risk Score (DTI + LTV)
if 'dti_risk_flag' in df_fe.columns and 'ltv_risk_flag' in df_fe.columns:
    risk_score_map = {'Low Risk': 1, 'Moderate Risk': 2, 'Elevated Risk': 3, 'High Risk': 4, 'Unknown': 2.5}
    
    dti_score = df_fe['dti_risk_flag'].map(risk_score_map)
    ltv_score = df_fe['ltv_risk_flag'].map(risk_score_map)
    
    df_fe['combined_risk_score'] = (dti_score + ltv_score) / 2
    print(f"\n✓ Combined Risk Score created (DTI + LTV average)")
    print(f"   Range: {df_fe['combined_risk_score'].min():.2f} - {df_fe['combined_risk_score'].max():.2f}")
    print(f"   Mean: {df_fe['combined_risk_score'].mean():.2f}")

CREATING RISK INDICATORS
✓ DTI Risk Flag already exists or DTI not available

✓ LTV Risk Flag created: 4 categories
ltv_risk_flag
Low Risk         160776
High Risk        127416
Elevated Risk    123131
Moderate Risk     82245
Name: count, dtype: int64

✓ Income Bracket created: 6 categories
income_bracket
$100-150K    121194
$150-250K     91948
$75-100K      91700
$50-75K       90185
>$250K        52998
<$50K         45543
Name: count, dtype: int64

✓ Loan Size Category created: 5 categories
loan_size_category
<$200K       125498
$200-300K    117110
$300-400K     96887
$400-600K     92712
>$600K        61361
Name: count, dtype: int64

✓ Combined Risk Score created (DTI + LTV average)
   Range: 1.00 - 4.00
   Mean: 2.36


## 3. Co-Borrower Features

EDA showed co-borrower presence is a strong predictor (92% vs 88% approval).

In [20]:
# ============================================================================
# CREATE CO-BORROWER FEATURES
# ============================================================================

print("="*80)
print("CREATING CO-BORROWER FEATURES")
print("="*80)

# 1. Co-borrower presence indicator using co_applicant_ethnicity
if 'co_applicant_ethnicity_1' in df_fe.columns:
    # Check for 'No co-applicant' value to identify solo applications
    df_fe['has_coborrower'] = (df_fe['co_applicant_ethnicity_1'] != 'No co-applicant').astype(int)
    
    # Count applications by co-borrower status
    n_with_coborrower = (df_fe['has_coborrower'] == 1).sum()
    n_without_coborrower = (df_fe['has_coborrower'] == 0).sum()
    
    print(f"✓ Co-borrower indicator created from co_applicant_ethnicity_1")
    print(f"   Applications WITH co-borrower: {n_with_coborrower:,} ({n_with_coborrower/len(df_fe):.1%})")
    print(f"   Applications WITHOUT co-borrower: {n_without_coborrower:,} ({n_without_coborrower/len(df_fe):.1%})")
    
    # Calculate approval rates by co-borrower status
    if 'target' in df_fe.columns:
        if n_with_coborrower > 0:
            approval_with = df_fe[df_fe['has_coborrower']==1]['target'].mean()
            print(f"   Approval rate WITH co-borrower: {approval_with:.2%}")
        else:
            print(f"   Approval rate WITH co-borrower: N/A (no applications)")
            
        if n_without_coborrower > 0:
            approval_without = df_fe[df_fe['has_coborrower']==0]['target'].mean()
            print(f"   Approval rate WITHOUT co-borrower: {approval_without:.2%}")
        else:
            print(f"   Approval rate WITHOUT co-borrower: N/A (no applications)")
    else:
        print("   ⚠ Warning: 'target' column not found - cannot calculate approval rates")
else:
    print("⚠ Warning: 'co_applicant_ethnicity_1' column not found")
    print("   Cannot create co-borrower indicator - this feature will be missing")
    df_fe['has_coborrower'] = 0  # Default to no co-borrower if column missing

# 2. Application type categorical
if 'applicant_sex' in df_fe.columns:
    df_fe['application_type'] = df_fe['applicant_sex'].map({
        'Male': 'Single - Male',
        'Female': 'Single - Female',
        'Joint': 'Joint',
        'Not Applicable': 'Joint'
    })
    df_fe['application_type'] = df_fe['application_type'].fillna('Unknown')
    print(f"\n✓ Application Type created: {df_fe['application_type'].nunique()} categories")
    print(df_fe.groupby('application_type')['target'].agg(['count', 'mean']))

CREATING CO-BORROWER FEATURES
✓ Co-borrower indicator created from co_applicant_ethnicity_1
   Applications WITH co-borrower: 226,116 (45.8%)
   Applications WITHOUT co-borrower: 267,452 (54.2%)
   Approval rate WITH co-borrower: 91.29%
   Approval rate WITHOUT co-borrower: 88.17%

✓ Application Type created: 3 categories
                   count      mean
application_type                  
Single - Female   180013  0.891597
Single - Male     290814  0.903602
Unknown            22741  0.832813


## 4. Geographic Features & Target Encoding

EDA showed significant geographic variation (78-94% approval by state). Use target encoding for high-cardinality features.

In [21]:
# ============================================================================
# TASK 3.2: GEOGRAPHIC FEATURES & TARGET ENCODING
# ============================================================================

print("="*80)
print("GEOGRAPHIC FEATURE ENGINEERING")
print("="*80)

# We'll use target encoding for high-cardinality geographic features
# Important: This should be done within cross-validation to avoid leakage
# For now, we'll create the encoding structure

# 1. State-level approval rate (low cardinality - safe to encode directly)
if 'state_code' in df_fe.columns:
    state_approval_rate = df_fe.groupby('state_code')['target'].mean()
    df_fe['state_approval_rate'] = df_fe['state_code'].map(state_approval_rate)
    print(f"✓ State approval rate encoded: {df_fe['state_code'].nunique()} states")
    print(f"   Range: {df_fe['state_approval_rate'].min():.2%} - {df_fe['state_approval_rate'].max():.2%}")

# 2. County-level features (high cardinality - mark for proper target encoding later)
if 'county_code' in df_fe.columns:
    n_counties = df_fe['county_code'].nunique()
    print(f"\n✓ County code: {n_counties} unique counties (will use target encoding in training)")
    
    # For now, create a frequency encoding as baseline
    county_freq = df_fe['county_code'].value_counts()
    df_fe['county_frequency'] = df_fe['county_code'].map(county_freq)
    print(f"   County frequency encoded (baseline): Mean {df_fe['county_frequency'].mean():.0f} applications per county")

# 3. MSA/MD features
if 'msa_md' in df_fe.columns:
    n_msa = df_fe['msa_md'].nunique()
    print(f"\n✓ MSA/MD code: {n_msa} unique metropolitan areas")
    
    # Frequency encoding
    msa_freq = df_fe['msa_md'].value_counts()
    df_fe['msa_frequency'] = df_fe['msa_md'].map(msa_freq)

# 4. Census tract income deviation (if tract median income available)
if 'tract_median_income' in df_fe.columns:
    df_fe['income_vs_tract'] = df_fe['income'] / df_fe['tract_median_income']
    df_fe['income_vs_tract'] = df_fe['income_vs_tract'].clip(upper=10)  # Cap extreme values
    print(f"\n✓ Income vs Tract Median: Created (shows if applicant income is above/below area median)")
    print(f"   Median ratio: {df_fe['income_vs_tract'].median():.2f}")

print("\n" + "="*80)
print("NOTE: Proper target encoding will be applied during model training")
print("to avoid data leakage (encoding must be fit only on training data)")
print("="*80)

GEOGRAPHIC FEATURE ENGINEERING
✓ State approval rate encoded: 53 states
   Range: 75.58% - 96.49%

✓ County code: 3078 unique counties (will use target encoding in training)
   County frequency encoded (baseline): Mean 1340 applications per county

NOTE: Proper target encoding will be applied during model training
to avoid data leakage (encoding must be fit only on training data)


## 5. Interaction Features

Create interaction terms between important features identified in EDA.

In [22]:
# ============================================================================
# TASK 3.3: CREATE INTERACTION FEATURES
# ============================================================================

print("="*80)
print("CREATING INTERACTION FEATURES")
print("="*80)

# 1. DTI × LTV (Combined financial stress)
if 'loan_to_income_ratio' in df_fe.columns and 'loan_to_value_ratio' in df_fe.columns:
    df_fe['dti_ltv_interaction'] = df_fe['loan_to_income_ratio'] * df_fe['loan_to_value_ratio'] / 100
    print(f"✓ DTI × LTV interaction created")
    print(f"   Median: {df_fe['dti_ltv_interaction'].median():.2f}")

# 2. Income × Has Co-borrower
if 'has_coborrower' in df_fe.columns:
    df_fe['income_coborrower_interaction'] = df_fe['income'] * df_fe['has_coborrower']
    print(f"\n✓ Income × Co-borrower interaction created")

# 3. Loan Amount × Interest Rate (Payment burden)
if 'interest_rate' in df_fe.columns:
    df_fe['loan_rate_interaction'] = (df_fe['loan_amount'] / 1000) * df_fe['interest_rate']
    print(f"\n✓ Loan Amount × Interest Rate interaction created")

# 4. Age × Income (Life-stage affordability)
if 'applicant_age' in df_fe.columns:
    # Convert age to numeric if it's categorical
    if df_fe['applicant_age'].dtype == 'object':
        age_mapping = {
            '<25': 22,
            '25-34': 30,
            '35-44': 40,
            '45-54': 50,
            '55-64': 60,
            '65-74': 70,
            '>74': 78
        }
        df_fe['applicant_age_numeric'] = df_fe['applicant_age'].map(age_mapping)
        df_fe['age_income_interaction'] = df_fe['applicant_age_numeric'] * (df_fe['income'] / 1000)
    else:
        df_fe['age_income_interaction'] = df_fe['applicant_age'] * (df_fe['income'] / 1000)
    
    print(f"\n✓ Age × Income interaction created")

print("\n" + "="*80)
print(f"Total features after engineering: {df_fe.shape[1]}")
print("="*80)

CREATING INTERACTION FEATURES
✓ DTI × LTV interaction created
   Median: 2.44

✓ Income × Co-borrower interaction created

✓ Loan Amount × Interest Rate interaction created

✓ Age × Income interaction created

Total features after engineering: 94


## 6. Feature Selection

Remove redundant and low-importance features before modeling.

In [23]:
# ============================================================================
# TASK 3.4: FEATURE SELECTION
# ============================================================================

print("="*80)
print("FEATURE SELECTION")
print("="*80)

# 1. Remove highly correlated features (>0.95)
print("\n1. Checking for highly correlated features...")

# Select only numeric columns for correlation analysis
numeric_cols = df_fe.select_dtypes(include=[np.number]).columns.tolist()
# Remove target from correlation analysis
if 'target' in numeric_cols:
    numeric_cols.remove('target')

corr_matrix = df_fe[numeric_cols].corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation > 0.95
high_corr_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

if high_corr_features:
    print(f"   Found {len(high_corr_features)} highly correlated features: {high_corr_features}")
    print(f"   Consider removing in final model to reduce multicollinearity")
else:
    print("   ✓ No features with correlation > 0.95")

# 2. Identify features to EXCLUDE (protected attributes)
print("\n2. Identifying protected attributes to EXCLUDE from modeling...")

protected_features = []
for col in df_fe.columns:
    col_lower = col.lower()
    if any(term in col_lower for term in ['race', 'ethnicity', 'sex', 'gender', 'minority', 'hispanic']):
        if col != 'has_coborrower':  # Keep this as it's a legitimate feature
            protected_features.append(col)

print(f"   Found {len(protected_features)} protected attributes:")
for feat in protected_features:
    print(f"   - {feat}")

# 3. Create lists of features for modeling
print("\n3. Categorizing features for modeling...")

# Engineered features to definitely include
engineered_features = [
    'loan_to_income_ratio', 'loan_to_value_ratio', 'housing_expense_ratio',
    'loan_costs_ratio', 'dti_risk_flag', 'ltv_risk_flag', 'income_bracket',
    'loan_size_category', 'combined_risk_score', 'has_coborrower',
    'state_approval_rate', 'county_frequency', 'income_vs_tract',
    'dti_ltv_interaction', 'income_coborrower_interaction', 'loan_rate_interaction'
]

# Keep only features that exist
engineered_features = [f for f in engineered_features if f in df_fe.columns]

print(f"   Engineered features: {len(engineered_features)}")

# Original important features from EDA
original_features = [
    'income', 'loan_amount', 'property_value', 'interest_rate', 'loan_term',
    'total_loan_costs', 'applicant_age', 'debt_to_income_ratio',
    'state_code', 'county_code', 'msa_md'
]
original_features = [f for f in original_features if f in df_fe.columns]

print(f"   Original features: {len(original_features)}")

# Create feature list for modeling
modeling_features = list(set(engineered_features + original_features))
modeling_features = [f for f in modeling_features if f not in protected_features]

print(f"\n   Total features for modeling: {len(modeling_features)}")
print(f"   Features excluded (protected): {len(protected_features)}")

FEATURE SELECTION

1. Checking for highly correlated features...
   Found 1 highly correlated features: ['age_income_interaction']
   Consider removing in final model to reduce multicollinearity

2. Identifying protected attributes to EXCLUDE from modeling...
   Found 18 protected attributes:
   - derived_ethnicity
   - derived_race
   - derived_sex
   - applicant_ethnicity_1
   - co_applicant_ethnicity_1
   - applicant_ethnicity_observed
   - co_applicant_ethnicity_observed
   - applicant_race_1
   - co_applicant_race_1
   - applicant_race_observed
   - co_applicant_race_observed
   - applicant_sex
   - co_applicant_sex
   - applicant_sex_observed
   - co_applicant_sex_observed
   - tract_minority_population_percent
   - race_minority
   - hispanic

3. Categorizing features for modeling...
   Engineered features: 15
   Original features: 10

   Total features for modeling: 25
   Features excluded (protected): 18


## 7. Final Data Preparation

Prepare data for train-test split and modeling.

In [24]:
# ============================================================================
# TASK 3.5: ENCODE CATEGORICAL VARIABLES
# ============================================================================

print("="*80)
print("ENCODING CATEGORICAL VARIABLES")
print("="*80)

# Create a copy for final processing
df_final = df_fe.copy()

# 1. Identify categorical features
categorical_features = df_final[modeling_features].select_dtypes(include=['object', 'category']).columns.tolist()
print(f"\nCategorical features to encode: {len(categorical_features)}")
for feat in categorical_features:
    print(f"  - {feat}: {df_final[feat].nunique()} unique values")

# 2. One-hot encode low-cardinality categoricals (<10 categories)
low_cardinality_cats = [col for col in categorical_features if df_final[col].nunique() < 10]
print(f"\n1. One-hot encoding {len(low_cardinality_cats)} low-cardinality features...")

if low_cardinality_cats:
    df_encoded = pd.get_dummies(df_final, columns=low_cardinality_cats, prefix=low_cardinality_cats, drop_first=True)
    print(f"   ✓ One-hot encoding complete")
    print(f"   New shape: {df_encoded.shape}")
else:
    df_encoded = df_final.copy()
    print(f"   No low-cardinality features to encode")

# 3. High-cardinality features (>10 categories) - save for target encoding during training
high_cardinality_cats = [col for col in categorical_features if df_final[col].nunique() >= 10]
print(f"\n2. High-cardinality features for target encoding: {len(high_cardinality_cats)}")
for feat in high_cardinality_cats:
    print(f"  - {feat}: {df_final[feat].nunique()} categories")

print("\n   Note: Target encoding will be applied during model training to prevent leakage")

print("\n" + "="*80)
print(f"Final dataset shape: {df_encoded.shape[0]:,} rows × {df_encoded.shape[1]} columns")
print("="*80)

ENCODING CATEGORICAL VARIABLES

Categorical features to encode: 8
  - county_code: 3078 unique values
  - debt_to_income_ratio: 19 unique values
  - loan_size_category: 5 unique values
  - income_bracket: 6 unique values
  - ltv_risk_flag: 4 unique values
  - dti_risk_flag: 5 unique values
  - applicant_age: 8 unique values
  - state_code: 53 unique values

1. One-hot encoding 5 low-cardinality features...
   ✓ One-hot encoding complete
   New shape: (493568, 112)

2. High-cardinality features for target encoding: 3
  - county_code: 3078 categories
  - debt_to_income_ratio: 19 categories
  - state_code: 53 categories

   Note: Target encoding will be applied during model training to prevent leakage

Final dataset shape: 493,568 rows × 112 columns


In [25]:
from sklearn.feature_extraction import FeatureHasher
import joblib

# ============================================================================
# FEATURE HASHING FOR HIGH-CARDINALITY FEATURES
# ============================================================================

print("="*80)
print("APPLYING FEATURE HASHING FOR HIGH-CARDINALITY FEATURES")
print("="*80)

# Dictionary to store hashers for inference
hashing_artifacts = {}

# 1. Hash state_code
if 'state_code' in df_encoded.columns:
    # Create feature hasher with 16 features (2^4 = 16 buckets)
    state_hasher = FeatureHasher(n_features=16, input_type='string')
    
    # Convert state_code to list of strings for hasher
    state_hashed = state_hasher.transform(df_encoded[['state_code']].astype(str).values).toarray()
    df_encoded['state_code_hashed'] = state_hashed.sum(axis=1)  # Aggregate hash values
    
    print(f"\n✓ State code hashed: {df_encoded['state_code'].nunique()} states → 16 hash features")
    print(f"   Hash distribution: min={df_encoded['state_code_hashed'].min():.2f}, max={df_encoded['state_code_hashed'].max():.2f}")
    print(f"   Hasher saved for inference")
    hashing_artifacts['state_code'] = {
        'hasher': state_hasher,
        'n_features': 16,
        'input_type': 'string',
        'output_column': 'state_code_hashed',
        'original_cardinality': df_encoded['state_code'].nunique()
    }
    
if 'debt_to_income_ratio' in df_encoded.columns:
    dti_hasher = FeatureHasher(n_features=5, input_type='string')
    dti_hashed = dti_hasher.transform(df_encoded[['debt_to_income_ratio']].astype(str).values).toarray()
    df_encoded['dti_hashed'] = dti_hashed.sum(axis=1)
    print(f"\n✓ DTI hashed: {df_encoded['debt_to_income_ratio'].nunique()} unique values → 5 hash features")
    print(f"   Hash distribution: min={df_encoded['dti_hashed'].min():.2f}, max={df_encoded['dti_hashed'].max():.2f}")
    hashing_artifacts['debt_to_income_ratio'] = {
        'hasher': dti_hasher,
        'n_features': 5,
        'input_type': 'string',
        'output_column': 'dti_hashed',
        'original_cardinality': df_encoded['debt_to_income_ratio'].nunique()
    }

# 2. Hash county_code (if present and high cardinality)
if 'county_code' in df_encoded.columns and df_encoded['county_code'].nunique() > 100:
    county_hasher = FeatureHasher(n_features=32, input_type='string')
    
    county_hashed = county_hasher.transform(df_encoded[['county_code']].astype(str).values).toarray()
    df_encoded['county_code_hashed'] = county_hashed.sum(axis=1)
    
    print(f"\n✓ County code hashed: {df_encoded['county_code'].nunique()} counties → 32 hash features")
    print(f"   Hash distribution: min={df_encoded['county_code_hashed'].min():.2f}, max={df_encoded['county_code_hashed'].max():.2f}")
    print(f"   Hasher saved for inference")
    hashing_artifacts['county_code'] = {
        'hasher': county_hasher,
        'n_features': 32,
        'input_type': 'string',
        'output_column': 'county_code_hashed',
        'original_cardinality': df_encoded['county_code'].nunique()
    }

print("\n" + "="*80)
print(f"Hashed features: {list(hashing_artifacts.keys())}")
print(f"Final dataset shape: {df_encoded.shape}")
print("="*80)

APPLYING FEATURE HASHING FOR HIGH-CARDINALITY FEATURES

✓ State code hashed: 53 states → 16 hash features
   Hash distribution: min=-1.00, max=1.00
   Hasher saved for inference

✓ DTI hashed: 19 unique values → 5 hash features
   Hash distribution: min=-1.00, max=1.00

✓ County code hashed: 3078 counties → 32 hash features
   Hash distribution: min=-1.00, max=1.00
   Hasher saved for inference

Hashed features: ['state_code', 'debt_to_income_ratio', 'county_code']
Final dataset shape: (493568, 115)


In [26]:
# ============================================================================
# UPDATE FEATURE LISTS AFTER ENCODING & HASHING
# ============================================================================

print("="*80)
print("UPDATING FEATURE LISTS AFTER ENCODING & HASHING")
print("="*80)

# 1. Get all columns after encoding and hashing
all_columns = df_encoded.columns.tolist()

# 2. Identify columns to exclude from modeling
exclude_from_modeling = set()

# Add target
if 'target' in all_columns:
    exclude_from_modeling.add('target')

# Add protected features
exclude_from_modeling.update(protected_features)

# Add original categorical columns that were one-hot encoded
exclude_from_modeling.update(low_cardinality_cats)

# Add original high-cardinality features that were hashed (keep only hashed versions)
for feature in hashing_artifacts.keys():
    if feature in all_columns:
        exclude_from_modeling.add(feature)

# Add any ID or metadata columns
metadata_cols = ['activity_year', 'lei', 'action_taken']
for col in metadata_cols:
    if col in all_columns:
        exclude_from_modeling.add(col)

print(f"\n1. Total columns after encoding: {len(all_columns)}")
print(f"2. Columns to exclude: {len(exclude_from_modeling)}")

# 3. Update modeling_features to include actual encoded columns
modeling_features = [col for col in all_columns if col not in exclude_from_modeling]

print(f"3. Modeling features after encoding: {len(modeling_features)}")

# 4. Identify new encoded features (one-hot encoded columns)
encoded_feature_cols = [col for col in all_columns 
                        if any(col.startswith(f"{cat}_") for cat in low_cardinality_cats)]

# 5. Identify hashed feature columns
hashed_feature_cols = [col for col in all_columns 
                       if col.endswith('_hashed')]

# 6. Update engineered_features to include all transformations
engineered_features_updated = []

# Keep original engineered features that are still present
for feat in engineered_features:
    if feat in all_columns and feat not in low_cardinality_cats:
        engineered_features_updated.append(feat)

# Add one-hot encoded features
engineered_features_updated.extend(encoded_feature_cols)

# Add hashed features
engineered_features_updated.extend(hashed_feature_cols)

# Remove duplicates
engineered_features = sorted(list(set(engineered_features_updated)))

print(f"4. Updated engineered features: {len(engineered_features)}")
print(f"   - One-hot encoded features: {len(encoded_feature_cols)}")
print(f"   - Hashed features: {len(hashed_feature_cols)}")

# Display sample of encoded features
if encoded_feature_cols:
    print(f"\n   Sample one-hot encoded features:")
    for feat in encoded_feature_cols[:5]:
        print(f"     - {feat}")
    if len(encoded_feature_cols) > 5:
        print(f"     ... and {len(encoded_feature_cols) - 5} more")

if hashed_feature_cols:
    print(f"\n   Hashed features created:")
    for feat in hashed_feature_cols:
        print(f"     - {feat}")

print("\n" + "="*80)
print(f"✅ Feature lists updated for modeling")
print(f"   Modeling Features: {len(modeling_features)}")
print(f"   Engineered Features: {len(engineered_features)}")
print("="*80)

UPDATING FEATURE LISTS AFTER ENCODING & HASHING

1. Total columns after encoding: 115
2. Columns to exclude: 30
3. Modeling features after encoding: 90
4. Updated engineered features: 39
   - One-hot encoded features: 25
   - Hashed features: 3

   Sample one-hot encoded features:
     - applicant_age_above_62
     - applicant_age_numeric
     - loan_size_category_$200-300K
     - loan_size_category_$300-400K
     - loan_size_category_$400-600K
     ... and 20 more

   Hashed features created:
     - state_code_hashed
     - dti_hashed
     - county_code_hashed

✅ Feature lists updated for modeling
   Modeling Features: 90
   Engineered Features: 39


In [27]:
# ============================================================================
# SAVE FEATURE-ENGINEERED DATA
# ============================================================================

print("="*80)
print("SAVING FEATURE-ENGINEERED DATA")
print("="*80)

# Save to processed directory
output_path = Path('data/processed/feature_engineered_data.csv')
output_path.parent.mkdir(parents=True, exist_ok=True)

# Save full dataset
df_encoded.to_csv(output_path, index=False)
print(f"\n✅ Feature-engineered data saved to: {output_path}")
print(f"   Shape: {df_encoded.shape[0]:,} rows × {df_encoded.shape[1]} columns")
print(f"   Size: {output_path.stat().st_size / 1024**2:.2f} MB")

# Save feature names for reference
feature_info = {
    'total_features': df_encoded.shape[1],
    'engineered_features': engineered_features,
    'original_features': original_features,
    'protected_features': protected_features,
    'high_cardinality_features': high_cardinality_cats,
    'modeling_features': modeling_features
}

import json
feature_info_path = output_path.parent / 'feature_engineering_info.json'
with open(feature_info_path, 'w') as f:
    # Convert non-serializable objects to lists
    serializable_info = {k: list(v) if isinstance(v, (list, set)) else v for k, v in feature_info.items()}
    json.dump(serializable_info, f, indent=2)

print(f"\n✅ Feature engineering info saved to: {feature_info_path}")

# Save hashing artifacts for inference
if 'hashing_artifacts' in locals() and hashing_artifacts:
    hashing_info_path = output_path.parent / 'hashing_artifacts.pkl'
    joblib.dump(hashing_artifacts, hashing_info_path)
    print(f"✅ Feature hashing artifacts saved to: {hashing_info_path}")
    
    # Save human-readable hashing config
    hashing_config = {
        feature: {
            'n_features': info['n_features'],
            'input_type': info['input_type'],
            'output_column': info['output_column'],
            'original_cardinality': info['original_cardinality']
        }
        for feature, info in hashing_artifacts.items()
    }
    
    hashing_config_path = output_path.parent / 'hashing_config.json'
    with open(hashing_config_path, 'w') as f:
        json.dump(hashing_config, f, indent=2)
    print(f"✅ Hashing configuration saved to: {hashing_config_path}")
    print(f"\n   To use during inference:")
    print(f"  >>> import joblib")
    print(f"  >>> hashers = joblib.load('{hashing_info_path.name}')")
    print(f"  >>> state_hasher = hashers['state_code']['hasher']")
    print(f"  >>> hashed_values = state_hasher.transform(new_data[['state_code']].astype(str).values).toarray()")

SAVING FEATURE-ENGINEERED DATA

✅ Feature-engineered data saved to: data/processed/feature_engineered_data.csv
   Shape: 493,568 rows × 115 columns
   Size: 695.52 MB

✅ Feature engineering info saved to: data/processed/feature_engineering_info.json
✅ Feature hashing artifacts saved to: data/processed/hashing_artifacts.pkl
✅ Hashing configuration saved to: data/processed/hashing_config.json

   To use during inference:
  >>> import joblib
  >>> hashers = joblib.load('hashing_artifacts.pkl')
  >>> state_hasher = hashers['state_code']['hasher']
  >>> hashed_values = state_hasher.transform(new_data[['state_code']].astype(str).values).toarray()


## 8. Feature Engineering Summary

Review of all feature engineering steps completed.

In [28]:
# ============================================================================
# FEATURE ENGINEERING SUMMARY
# ============================================================================

print("="*80)
print("FEATURE ENGINEERING SUMMARY")
print("="*80)

print("\n✅ PHASE 3 COMPLETE: Feature Engineering")
print("\n" + "="*80)
print("TASKS COMPLETED:")
print("="*80)

print("\n1. ✓ Underwriting Metrics Created:")
print("   - Loan-to-Income Ratio")
print("   - Loan-to-Value Ratio (LTV)")
print("   - Housing Expense Ratio")
print("   - Loan Costs Ratio")

print("\n2. ✓ Risk Indicators Created:")
print("   - DTI Risk Flag (4-tier)")
print("   - LTV Risk Flag (4-tier)")
print("   - Income Bracket (6 categories)")
print("   - Loan Size Category (5 categories)")
print("   - Combined Risk Score (DTI + LTV)")

print("\n3. ✓ Co-Borrower Features Created:")
print("   - Co-borrower presence indicator")
print("   - Application type (Single vs Joint)")

print("\n4. ✓ Geographic Features Created:")
print("   - State approval rate encoding")
print("   - County frequency encoding")
print("   - Income vs Tract Median ratio")

print("\n5. ✓ Interaction Features Created:")
print("   - DTI × LTV interaction")
print("   - Income × Co-borrower interaction")
print("   - Loan Amount × Interest Rate interaction")
print("   - Age × Income interaction")

print("\n6. ✓ Feature Selection Completed:")
print(f"   - {len(modeling_features)} features selected for modeling")
print(f"   - {len(protected_features)} protected attributes excluded")
print(f"   - {len(high_cardinality_cats)} high-cardinality features marked for target encoding")

print("\n7. ✓ Categorical Encoding:")
print(f"   - {len(low_cardinality_cats)} features one-hot encoded")
print(f"   - {len(high_cardinality_cats)} features prepared for target encoding")

print("\n" + "="*80)
print("NEXT STEPS (Phase 4):")
print("="*80)
print("\n1. Train-Test-Validation Split (70-15-15)")
print("2. Apply target encoding on training data only")
print("3. Feature scaling/normalization")
print("4. Baseline model training (Logistic Regression)")
print("5. Advanced models (XGBoost, Random Forest, Neural Network)")
print("6. Model evaluation and comparison")
print("7. Fairness auditing on test set")

print("\n" + "="*80)
print(f"Dataset ready for modeling: {output_path}")
print("="*80)

FEATURE ENGINEERING SUMMARY

✅ PHASE 3 COMPLETE: Feature Engineering

TASKS COMPLETED:

1. ✓ Underwriting Metrics Created:
   - Loan-to-Income Ratio
   - Loan-to-Value Ratio (LTV)
   - Housing Expense Ratio
   - Loan Costs Ratio

2. ✓ Risk Indicators Created:
   - DTI Risk Flag (4-tier)
   - LTV Risk Flag (4-tier)
   - Income Bracket (6 categories)
   - Loan Size Category (5 categories)
   - Combined Risk Score (DTI + LTV)

3. ✓ Co-Borrower Features Created:
   - Co-borrower presence indicator
   - Application type (Single vs Joint)

4. ✓ Geographic Features Created:
   - State approval rate encoding
   - County frequency encoding
   - Income vs Tract Median ratio

5. ✓ Interaction Features Created:
   - DTI × LTV interaction
   - Income × Co-borrower interaction
   - Loan Amount × Interest Rate interaction
   - Age × Income interaction

6. ✓ Feature Selection Completed:
   - 90 features selected for modeling
   - 18 protected attributes excluded
   - 3 high-cardinality features marked